In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Exercise Recommendation Model 1

In [11]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
df = pd.read_csv('/content/exercise_dataset.csv')

In [13]:
print(df.head())

   UserID  Age  Gender              Injury Fitness Level  ExerciseID  \
0       1   55  Female        Quadriplegia      Beginner           8   
1       2   55    Male  Multiple Sclerosis      Beginner           4   
2       3   86    Male        Quadriplegia  Intermediate           5   
3       4   51  Female          Paraplegia      Beginner           1   
4       5   43    Male  Spinal Cord Injury  Intermediate           6   

       Exercise Name  
0  Neck Side Stretch  
1       Neck Flexion  
2    Neck Isometrics  
3         Chin Tucks  
4  Seated Head Turns  


In [14]:
print('Exercises:\n')

for exercise_name in df['Exercise Name'].unique():
    print(exercise_name)

Exercises:

Neck Side Stretch
Neck Flexion
Neck Isometrics
Chin Tucks
Seated Head Turns
Neck Rotation
Neck Extension
Neck Side Bend
Shoulder Shrugs


In [15]:
# Recommender class
class Recommender:
    def __init__(self):
        self.scaler = MinMaxScaler()
        self.gender_mapping = {"Male": 1, "Female": 0}
        self.injury_mapping = {"Quadriplegia": 0, "Multiple Sclerosis": 1, "Paraplegia": 2, "Spinal Cord Injury": 3, "Amputation": 4}
        self.fitness_mapping = {"Beginner": 1, "Intermediate": 2, "Advanced": 3}
        self.rep_counts = {"Beginner": 6, "Intermediate": 10, "Advanced": 15}

    def preprocess_data(self, data):
        data = data.copy()
        data["Gender"] = data["Gender"].map(self.gender_mapping)
        data["Injury"] = data["Injury"].map(self.injury_mapping)
        data["Fitness Level"] = data["Fitness Level"].map(self.fitness_mapping)
        data[["Age"]] = self.scaler.transform(data[["Age"]])
        return data

    def fit(self, data):
        data = data.copy()
        self.scaler.fit(data[["Age"]])
        return self

    def create_user_profile(self, user_age, user_gender, user_injury, user_fitness_level):
        user_data = pd.DataFrame({
            "Age": [user_age],
            "Gender": [self.gender_mapping.get(user_gender, 0)],
            "Injury": [self.injury_mapping.get(user_injury, 0)],
            "Fitness Level": [self.fitness_mapping.get(user_fitness_level, 1)]
        })
        user_data[["Age"]] = self.scaler.transform(user_data[["Age"]])
        return user_data.values.reshape(1, -1)

    def recommend_exercises(self, data, user_profile, fitness_level, top_n=5):
        exercise_vectors = data[["Age", "Gender", "Injury", "Fitness Level"]].values
        similarities = cosine_similarity(user_profile, exercise_vectors).flatten()
        data["Similarity"] = similarities
        exercise_summary = (
            data.groupby("Exercise Name")
            .agg({"Similarity": "mean"})
            .reset_index()
        )
        recommendations = exercise_summary.sort_values(by="Similarity", ascending=False).head(top_n)
        recommended_reps = self.rep_counts.get(fitness_level, 6)
        return [{"Exercise Name": row["Exercise Name"], "Recommended Reps": recommended_reps}
                for _, row in recommendations.iterrows()]

In [16]:
# Training function
def train_recommender1():

    recommender = Recommender()
    recommender.fit(df)

    with open("recommender1.pkl", "wb") as f:
        pickle.dump(recommender, f)

train_recommender1()

In [18]:
# test

gender = "Male"
age = 30
injury = "Amputation"
fitness_level = "Beginner"

# Load the trained recommender from the pickle file
with open("recommender1.pkl", "rb") as f:
    recommender = pickle.load(f)

processed_df = recommender.preprocess_data(df)
user_profile = recommender.create_user_profile(age, gender, injury, fitness_level)
recommendations = recommender.recommend_exercises(processed_df, user_profile, fitness_level)


for rec in recommendations:
        print(f"- {rec['Exercise Name']}: {rec['Recommended Reps']} reps")

- Neck Extension: 6 reps
- Chin Tucks: 6 reps
- Seated Head Turns: 6 reps
- Neck Side Bend: 6 reps
- Shoulder Shrugs: 6 reps
